참고링크 https://bashtage.github.io/linearmodels/iv/examples/basic-examples.html

In [9]:
import numpy as np
from linearmodels.datasets import mroz
from statsmodels.api import add_constant

print(mroz.DESCR)
data = mroz.load()
data = data.dropna()
data = add_constant(data, has_constant="add")


T.A. Mroz (1987), "The Sensitivity of an Empirical Model of Married Women's
Hours of Work to Economic and Statistical Assumptions," Econometrica 55,
765-799.

nlf        1 if in labor force, 1975
hours      hours worked, 1975
kidslt6    # kids < 6 years
kidsge6    # kids 6-18
age        woman's age in yrs
educ       years of schooling
wage       estimated wage from earns., hours
repwage    reported wage at interview in 1976
hushrs     hours worked by husband, 1975
husage     husband's age
huseduc    husband's years of schooling
huswage    husband's hourly wage, 1975
faminc     family income, 1975
mtr        fed. marginal tax rate facing woman
motheduc   mother's years of schooling
fatheduc   father's years of schooling
unem       unem. rate in county of resid.
city       =1 if live in SMSA
exper      actual labor mkt exper
nwifeinc   (faminc - wage*hours)/1000
lwage      log(wage)
expersq    exper^2



In [2]:
from linearmodels.iv import IV2SLS

res_ols = IV2SLS(np.log(data.wage), data[["const", "educ"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                   wage   R-squared:                      0.1179
Estimator:                        OLS   Adj. R-squared:                 0.1158
No. Observations:                 428   F-statistic:                    57.196
Date:                Mon, Oct 10 2022   P-value (F-stat)                0.0000
Time:                        20:16:36   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.1852     0.1848    -1.0022     0.3163     -0.5474      0.1770
educ           0.1086     0.0144     7.5628     0.00

In [3]:
res_first = IV2SLS(data.educ, data[["const", "fatheduc"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_first)
data["educ_hat"] = data.educ - res_first.resids

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.1726
Estimator:                        OLS   Adj. R-squared:                 0.1706
No. Observations:                 428   F-statistic:                    89.258
Date:                Mon, Oct 10 2022   P-value (F-stat)                0.0000
Time:                        20:16:43   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          10.237     0.2753     37.186     0.0000      9.6975      10.777
fatheduc       0.2694     0.0285     9.4476     0.00

In [4]:
res_second = IV2SLS(np.log(data.wage), data[["const"]], data.educ, data.fatheduc).fit(
    cov_type="unadjusted"
)
print(res_second)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.0934
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0913
No. Observations:                 428   F-statistic:                    2.8487
Date:                Mon, Oct 10 2022   P-value (F-stat)                0.0914
Time:                        20:16:50   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.4411     0.4451     0.9911     0.3216     -0.4312      1.3134
educ           0.0592     0.0351     1.6878     0.09

In [5]:
from linearmodels.iv import compare

res_direct = IV2SLS(np.log(data.wage), data[["const", "educ_hat"]], None, None).fit(
    cov_type="unadjusted"
)
print(compare({"OLS": res_ols, "2SLS": res_second, "Direct": res_direct}))

                         Model Comparison                        
                                OLS           2SLS         Direct
-----------------------------------------------------------------
Dep. Variable                  wage           wage           wage
Estimator                       OLS        IV-2SLS            OLS
No. Observations                428            428            428
Cov. Est.                unadjusted     unadjusted     unadjusted
R-squared                    0.1179         0.0934         0.0060
Adj. R-squared               0.1158         0.0913         0.0037
F-statistic                  57.196         2.8487         2.5982
P-value (F-stat)          3.941e-14         0.0914         0.1070
==================     ============   ============   ============
const                       -0.1852         0.4411         0.4411
                          (-1.0022)       (0.9911)       (0.9465)
educ                         0.1086         0.0592               
          